In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

#sykey-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import string
import pandas as pd

import datetime
from pandas.tseries.offsets import BDay #business day

In [ ]:
def cleanText(row):
    clean = row['raw']
    clean = clean.lower()

    # remove punctuation + unwanted char
    clean2 = ''
    for char in clean:
        if char in "\n\t":
            clean2 = clean2 + ' '
        elif char not in string.punctuation + "’":
            clean2 = clean2 + char

    #print(clean2)
    
    ps = nltk.PorterStemmer()
    wn = nltk.WordNetLemmatizer()
    
    words = clean2.split(' ')
    clean_words = []
    # remove stopwords
    for word in words:
        if word not in nltk.corpus.stopwords.words('english'):
            word = ps.stem(word)
            word = wn.lemmatize(word)
            clean_words.append(word)
            
    return ' '.join(clean_words)


In [ ]:
text = {
    'raw': 'Getty Images\n\nImagine opening: electricity feet Instagram not to post a photo or comment on someone’s story but instead to pay for your groceries.'
}
cleanText(text)

In [ ]:
def make_clean_column(df):
    df['clean'] = df.apply(cleanText, axis=1) #apply() : one by one

In [ ]:
# word vectorzie TF-idf
# TfidfVectorizer

def vectorize(df):
    tfidf_vec = TfidfVectorizer(analyzer='word', stop_words=nltk.corpus.stopwords.words('english'))
    data = tfidf_vec.fit_transform(df['clean'])
   
    df2 = pd.DataFrame(data.toarray(), columns=tfidf_vec.get_feature_names())
    return df2

In [ ]:
articles = [
    'getti imag imagin open electr foot',
    'admiss dropbox cofoud ceo drew houston spent',
    'instagram'
]
test = pd.DataFrame(articles, columns=['clean'])
vectorize(test)

In [ ]:
def find_price_from_date(td, df):
    datestring = td.strftime('%Y-%m-%d')
    print(datestring)
    row = dfp.oc [df['Date'] == datestrin]
    if row.shape[0]== 0:
        return None
    
    return row['Adj Close'].iloc[0]

In [ ]:
# df = pd.read_csv('price_AMZN.csv')
# datestring = '2020-05-12T19:20:41+:00:00'
# td = datetime.datetime.fromisoformat(datestring)

# find_price_from_date(td, df)

In [ ]:
def find_price_from_date(td, dfp):
    datestring = td.strftime('%Y-%m-%d')
    print(datestring)
    row = dfp.loc [dfp['Date'] == datestring]
    
    if row.shape[0] == 0:
        return None
    return row['Adj Close'].iloc[0]

In [ ]:
# dfp = pd.read_csv("price_AMZN.csv")
# td = datetime.datetime.fromisoformat("2020-05-06T20:17:57+:00:00")
# find_price_from_date(td, dfp)
dfp = pd.read_csv("price_AMZN.csv")
td = datetime.datetime.fromisoformat("2020-05-06T20:17:57+00:00")
find_price_from_date(td, dfp)

In [ ]:
def is_big_event(datestring, df):
    dfp = pd.read_csv('price_AMZN.csv')
    
    td = datetime.datetime.fromisoformat(datestring)
    t_price = find_price_from_date(td, dfp)
    
    yd = td - BDay(1)
    y_price = find_price_from_date(yd, dfp)
    
    if t_price == None or y_price == None:
        return 0
    
    delta = abs((t_price - y_price)/y_price)
    
    print("T:%s Y:%s => %s", t_price, y_price, delta)
    
    if (delta > 0.01):
        return 1
    else: 
        return 0
    
    #iso-8601 standard - date format 

In [ ]:
# test is_big_event()
# df = pd.read_csv("AMZN_STEP1.csv")
# is_big_event("2020-05-11T19:20:41+00:00", df)

In [ ]:
# def make_label_column(df):
#     for idx, row in df.iterrows():
#         res = is_big_event(row['data'], df)
#         print("%s %s %s => %s" % (idx, row['date'], row['symbol'], res))
#         df.loc[idx, '@@label'] = res
def make_label_column(df):
    for idx, row in df.iterrows():
        res = is_big_event(row['date'], df)
        print("%s %s %s => %s" % (idx, row['date'], row['symbol'], res))
        df.loc[idx, '@@label'] = res

In [ ]:
# df = pd.read_csv('AMZN_STEP1.csv')
# make_label_column(df)
df = pd.read_csv('AMZN_STEP1.csv')
make_label_column(df)

In [ ]:
def main():
    def main():
    df = pd.read_csv('AMZN_STEP1.csv')
    make_clean_column(df)
    make_label_column(df)
    dfv = vectorize(df)
    print(dfv.info())
    
    df.drop(columns=['symbol', 'raw', 'clean'], inplace=True)
    df.rename(columns={'url': '@@url', 'date': '@@date'}, inplace=True)    
    
    dfm = pd.concat([df, dfv], axis=1)
    dfm.to_csv('AMZN_STEP2.csv')

    return dfm

    
main()